In [9]:
import amazon_reviews
import openai
import json
import os

openai.api_key  = '....'

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [10]:
amazon, amazon_value = amazon_reviews.search()

b = amazon_value

json_file = amazon

with open(json_file, "r") as file:
    data = json.load(file)

reviews = []

for review_list in data:
    for review in review_list:
        reviews.append(review['body'])


In [11]:
prompt = f"""

Analyze the following collection of reviews and employ topic modeling techniques to categorize the feedback into specific features of the product.
Divide each feature in positive characteristics and in negative characteristics.
Response format: 
                Feature:       
                    -Name: x
                    -Positive characteristics: y (your own words)
                    -Negative characteristics: z (your own words)
                    -If there are no positive or negative characteristics, write "Not applicable".

Provide it in JSON format to save in JSON file.

Review text: '''{reviews}'''
"""
response = get_completion(prompt)

# Save response to a JSON file
with open('response.json', 'w', encoding='utf-8') as f:
    f.write(response)

print(response, "\n")

{
    "Features": [
        {
            "Name": "Camera",
            "Positive characteristics": "Top-notch video and camera quality, photos turn out phenomenal, stable videos, high-quality sound from speakers",
            "Negative characteristics": "Not applicable"
        },
        {
            "Name": "Battery",
            "Positive characteristics": "Good battery life, lasts about 14 hours",
            "Negative characteristics": "Battery is okay, not great"
        },
        {
            "Name": "Box condition",
            "Positive characteristics": "Great saving on price compared to other outlets",
            "Negative characteristics": "Box may be damaged"
        },
        {
            "Name": "Performance",
            "Positive characteristics": "Fast",
            "Negative characteristics": "Phone constantly lags, slow to respond to commands, frequently turns on and off by itself, touch screen is unresponsive, frustrating to use, gets warm easily"
        },

In [12]:
import json
import dominate
from dominate.tags import *

# Load your JSON data
with open('response.json', 'r') as f:
    data = json.load(f)

doc = dominate.document(title='Features')

with doc.head:
    style("""
        body {
            font-family: Arial, sans-serif;
        }
        .feature {
            border: 1px solid #ddd;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 5px;
        }
        .feature-name {
            font-size: 20px;
            color: #444;
        }
        .positive {
            color: green;
        }
        .negative {
            color: red;
        }
    """)

with doc:
    h1(f'Product ID: {amazon_value}', style="text-align:center;")

    for feature in data['Features']:
        with div(cls='feature'):
            h2(feature['Name'], cls='feature-name')
            with div(cls='positive'):
                h3('Positive characteristics:')
                ul(li(pc) for pc in [feature['Positive characteristics']])   # Updated this line
            with div(cls='negative'):
                h3('Negative characteristics:')
                ul(li(nc) for nc in [feature['Negative characteristics']])   # Updated this line


with open('output.html', 'w') as f:
    f.write(str(doc))